# 机器学习100天——第3天：多元线性回归（Multiple Linear Regression）

## 第1步：数据预处理

**导入库**

In [1]:
import pandas as pd
import numpy as np

**导入数据集**

In [2]:
dataset = pd.read_csv('../datasets/50_Startups.csv')
X = dataset.iloc[ : , :-1].values
Y = dataset.iloc[ : ,  4 ].values

**将类别数据数字化**

In [3]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder = LabelEncoder()
X[: , 3] = labelencoder.fit_transform(X[ : , 3])
onehotencoder = OneHotEncoder(categorical_features = [3])
X = onehotencoder.fit_transform(X).toarray()

**躲避虚拟变量陷阱**

In [4]:
X = X[: , 1:]

**拆分数据集为训练集和测试集**

In [5]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

/home/ymao/usr/miniconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 第2步：在训练集上训练多元线性回归模型

In [6]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

## 第3步：在测试集上预测结果

In [7]:
y_pred = regressor.predict(X_test)

In [8]:
print(y_pred)

[ 103015.20159796  132582.27760815  132447.73845175   71976.09851258
  178537.48221056  116161.24230166   67851.69209676   98791.73374687
  113969.43533013  167921.06569551]


<b>完整的项目请前往Github项目<a href="https://github.com/MachineLearning100/100-Days-Of-ML-Code">100-Days-Of-ML-Code</a>查看。有任何的建议或者意见欢迎在issue中提出~</b>